# Clean Data

## Imports

In [1]:
"""Module providing data cleaning and export."""

import pandas as pd
import numpy as np

## Functions

In [2]:
# Replace ',' with '.' and convert to float
def coma_a_punto(x):
    """
    Función para convertir la ',' a '.'

    Parameters:
    --------------
    x : string to be converted

    """
    x = x.str.replace(',', '.').astype(float)
    return x

## Load Data with Correct Data Types

In [3]:
to_remove = [
    'Código', 'Centro', 'ID (5)', 'Fecha_Nac', 'Edad_agrupada', 'Grupo_edad',
    'Puntos_EQ1', 'Puntos_EQ2', 'Puntos_EQ3', 'Puntos_Andar', 'Puntos_Silla',
]

In [4]:
data_types = {
    'ID (3)': pd.Int64Dtype(),
    'Sexo': pd.Int64Dtype(),
    'Edad': pd.Int64Dtype(),
    'Hand_Izq': pd.Int64Dtype(),
    'Hand_Drch': pd.Int64Dtype(),
    'MQI': pd.StringDtype(), # convert
    'Cut-off points 1=NORMAL;2=BAJA;3=POBRE)': pd.Int64Dtype(),
    'Músculo relativ': pd.StringDtype(), # convert
    'Altura': pd.StringDtype(), # convert
    'Peso': pd.Int64Dtype(),
    'IMC': pd.Int64Dtype(),
    'Grasa': pd.StringDtype(), # errores en linea 192, replace "#NULL!" with average?
    'Músculo': pd.Int64Dtype(),
    'EQ1': pd.StringDtype(), # convert
    'EQ2': pd.StringDtype(), # convert
    'EQ3': pd.StringDtype(), # convert
    'Agil_m_s': pd.StringDtype(), # convert
    'Sarcopenia 1=SI;0=NO v/m': pd.Int64Dtype(),
    'Sarcopenia SI/No handgrip':  pd.Int64Dtype(),
    'sarcEWGSOP 0=NO, 1=moderada, 2=severa': pd.Int64Dtype(),
    'Andar_Agil4m':  pd.StringDtype(), # convert
    'Sentadilla': pd.StringDtype(),
    'Mean Power': pd.StringDtype(), # convert
    'Relative power': pd.StringDtype(),  # convert
    'SPPB': pd.Int64Dtype(),
    'Puntos_EQ(suma de todos los test equilibrio)': pd.Int64Dtype(),
    'Fragilidad(SPPB)': pd.Int64Dtype(),
    'gruposbuena': pd.Int64Dtype()
}

data = pd.read_csv("../data/datos_completos.csv", dtype=data_types)
print(data.shape)

(1253, 28)


## Simplify Column Names

In [5]:
# Standardize column names
data.columns = (
        data.columns
        .str.replace(' ', '_')
        .str.lower()
    )

In [6]:
# Rename Columns
columns = ['id_3', 'sexo', 'edad', 'hand_izq', 'hand_drch', 'mqi',
       'cut_off_points', 'músculo_relativ', 'altura', 'peso', 'imc',
       'grasa', 'músculo', 'eq1', 'eq2', 'eq3', 'agil_m_s',
       'sarcopenia_v', 'sarcopenia_handgrip', 'sarcewgsop',
       'andar_agil4m', 'sentadilla', 'mean_power', 'relative_power',
       'sppb', 'pnts_eq', 'fragilidad', 'gruposbuena']

data = data.set_axis(columns, axis=1)
print(data.shape)

(1253, 28)


# Clean Data

## Mean Power and Relative Power Columns

### Datos puros

In [7]:
print(f"Filas totales antes: {data.shape[0]}")

# Remove rows with division errors
datos_puros = data.drop(data.loc[data['mean_power'] == '#DIV/0!'].index)
num_of_files_with_div_error = datos_puros[datos_puros['mean_power'] == '#DIV/0!'].size

print(f"Num of files w/ division error: {num_of_files_with_div_error}")
print(f"Filas totales después: {datos_puros.shape[0]}")

Filas totales antes: 1253
Num of files w/ division error: 0
Filas totales después: 1251


In [8]:
# Correct the data type
datos_puros[['mean_power']] = datos_puros[['mean_power']].apply(coma_a_punto)

### Datos modificados

In [9]:
# Fill in the 'mean_power' column for the modified data
datos_modificados = data.copy()

# Substitute '#DIV/0!' with nan
datos_modificados[['mean_power']] = datos_modificados[['mean_power']].replace('#DIV/0!', np.nan)

# Verify success
print(f"Rows containing nan: {datos_modificados['mean_power'].isnull().sum()}")
print(f"Rows still containing '#DIV/0!': {data[datos_modificados['mean_power'] == '#DIV/0!'].size}")

# Convert 'mean_power' column to float
datos_modificados[['mean_power']] = datos_modificados[['mean_power']].apply(coma_a_punto)

# Compute average of column
mean = round(datos_modificados['mean_power'].mean(), 2)
print(f"Average mean_power: {mean}")

# Substitute na with column average
datos_modificados['mean_power'] = datos_modificados['mean_power'].fillna(mean)

print(f"Rows containing nan after change: {datos_modificados['mean_power'].isnull().sum()}")

Rows containing nan: 2
Rows still containing '#DIV/0!': 0
Average mean_power: 121.71
Rows containing nan after change: 0


In [10]:
# Substitute with nan
datos_modificados[['relative_power']] = datos_modificados[['relative_power']]\
                                        .replace('#DIV/0!', np.nan)

# Verify the number of rows with nan before
print(f"Numero de filas con nan before: {datos_modificados['relative_power'].isnull().sum()}")

# Fill rows with recalculated values
datos_modificados['relative_power'] = datos_modificados.apply(lambda x: \
                                                    str(round(x['mean_power']/x['peso'],2))\
                                                              .replace('.',',')\
                                                              if pd.isnull(x['relative_power'])\
                                                              else x['relative_power'], axis=1)

# Verify the number of rows with nan after
print(f"Numero de filas con nan after change: {datos_modificados['relative_power'].isnull().sum()}")

Numero de filas con nan before: 2
Numero de filas con nan after change: 0


## Float Conversiones

In [11]:
# List of columns that need to change to float
float_columns = ['relative_power', 'mqi', 'músculo_relativ', 'altura', 'agil_m_s',
                 'andar_agil4m', 'sentadilla', 'eq1', 'eq2', 'eq3']

# Apply change to pure data
datos_puros[float_columns] = datos_puros[float_columns].apply(coma_a_punto)

print(datos_modificados.loc[1071]['relative_power'])

# Apply change to modified data data
datos_modificados[float_columns] = datos_modificados[float_columns].apply(coma_a_punto)

2,3


### Change row with typing error

In [12]:
# Correct row where eq_3 is 110
print(f"Number of rows with value error in eq3: {datos_puros[ datos_puros['eq3'] > 10 ].shape[0]}")
replace_this = datos_puros[ datos_puros['eq3'] > 10 ]['eq3']
print(f"Value to replace: {replace_this}")

datos_puros['eq3'] = datos_puros.apply(lambda x: 10 if x['eq3'] > 10 else x['eq3'], axis=1)
datos_modificados['eq3'] = datos_modificados.apply(lambda x: 10 if x['eq3'] > 10 \
                                                   else x['eq3'], axis=1
                                                  )

print("Number of rows in pure data with value error in eq3:")
print(f"{datos_puros[ datos_puros['eq3'] > 10 ].shape[0]}")
print("Number of rows in modified data with value error in eq3:")
print(f"{datos_modificados[ datos_modificados['eq3'] > 10 ].shape[0]}")

Number of rows with value error in eq3: 1
Value to replace: 346    110.0
Name: eq3, dtype: float64
Number of rows in pure data with value error in eq3:
0
Number of rows in modified data with value error in eq3:
0


## Grasa  Columna

### Datos Puros

In [13]:
print(f"Numero de filas en total: {datos_puros.shape[0]}")
print("Numero de filas con datos perdidos antes del cambio:")
print(f"{datos_puros[datos_puros['grasa'] == '#NULL!'].shape[0]}")

Numero de filas en total: 1251
Numero de filas con datos perdidos antes del cambio:
12


In [14]:
# Soltar filas con datos faltantes
datos_puros = datos_puros.drop(datos_puros.loc[datos_puros['grasa'] == '#NULL!'].index)

print(f"Numero de filas en total: {datos_puros.shape[0]}")
print("Numero de filas con datos perdidos después:")
print(f"{datos_puros[datos_puros['grasa'] == '#NULL!'].shape[0]}")

Numero de filas en total: 1239
Numero de filas con datos perdidos después:
0


In [15]:
# Corregir el tipo de datos
datos_puros['grasa'] = datos_puros['grasa'].astype('Int64')

### Datos Modificados

In [16]:
# Investigate errorss in the data in the 'grasa' column
print(f"Filas con null valores: {datos_modificados['grasa'].isnull().sum()}")
print(f"Filas con '#NULL!': {datos_modificados[datos_modificados['grasa'] == '#NULL!'].shape[0]}")

# Substitute '#NULL!' with nan
datos_modificados[['grasa']] = datos_modificados[['grasa']].replace('#NULL!', np.nan)
print("\n---- Change Applied ----\n")

# Verify success
print(f"Filas con nan: {datos_modificados['grasa'].isnull().sum()}")
print(f"Filas con '#NULL!': {datos_modificados[datos_modificados['grasa'] == '#NULL!'].size}")

Filas con null valores: 0
Filas con '#NULL!': 12

---- Change Applied ----

Filas con nan: 12
Filas con '#NULL!': 0


In [17]:
# Convert grasa column to Int
datos_modificados['grasa'] = datos_modificados['grasa'].astype('Int64')

# Compute mean of grasa column
mean = int(datos_modificados['grasa'].mean())
print(mean)

# Substitute na with average
datos_modificados['grasa'] = datos_modificados['grasa'].fillna(mean)

38


In [18]:
# Number of rows with missing values in column 'grasa'
print("Numero de filas con '#NULL!':")
print(f"{datos_modificados[datos_modificados['grasa'] == '#NULL!'].shape[0]}")
print("Numero de filas con datos faltantes:")
print(f"{datos_modificados[datos_modificados['grasa'] == 'NaN'].shape[0]}")

# Check some of the rows that had 'nan'
print(datos_modificados.loc[192:198,'grasa'])

Numero de filas con '#NULL!':
0
Numero de filas con datos faltantes:
0
192    38
193    38
194    38
195    38
196    38
197    38
198    38
Name: grasa, dtype: Int64


## Comprobar Cambios

In [19]:
# Number of rows in datos_puros
print(f"Numero de filas en total en datos puros: {datos_puros.shape[0]}\n")

# Verify column names and data types are correct
datos_puros.info()

Numero de filas en total en datos puros: 1239

<class 'pandas.core.frame.DataFrame'>
Index: 1239 entries, 0 to 1252
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id_3                 1239 non-null   Int64  
 1   sexo                 1239 non-null   Int64  
 2   edad                 1239 non-null   Int64  
 3   hand_izq             1239 non-null   Int64  
 4   hand_drch            1239 non-null   Int64  
 5   mqi                  1239 non-null   float64
 6   cut_off_points       1239 non-null   Int64  
 7   músculo_relativ      1239 non-null   float64
 8   altura               1239 non-null   float64
 9   peso                 1239 non-null   Int64  
 10  imc                  1239 non-null   Int64  
 11  grasa                1239 non-null   Int64  
 12  músculo              1239 non-null   Int64  
 13  eq1                  1239 non-null   float64
 14  eq2                  1239 non-null   float64
 

In [20]:
# Number of rows in datos_modificados
print(f"Numero de filas en total en los datos modificados: {datos_modificados.shape[0]}\n")

# Verify column names and data types are correct
datos_modificados.info()

Numero de filas en total en los datos modificados: 1253

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253 entries, 0 to 1252
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id_3                 1253 non-null   Int64  
 1   sexo                 1253 non-null   Int64  
 2   edad                 1253 non-null   Int64  
 3   hand_izq             1253 non-null   Int64  
 4   hand_drch            1253 non-null   Int64  
 5   mqi                  1253 non-null   float64
 6   cut_off_points       1253 non-null   Int64  
 7   músculo_relativ      1253 non-null   float64
 8   altura               1253 non-null   float64
 9   peso                 1253 non-null   Int64  
 10  imc                  1253 non-null   Int64  
 11  grasa                1253 non-null   Int64  
 12  músculo              1253 non-null   Int64  
 13  eq1                  1253 non-null   float64
 14  eq2                  1253 non-n

# Exportar Datos Limpios

In [21]:
# Save Cleaned data
datos_puros.to_csv("../data/datos_puros.csv", index=False)

In [22]:
# Save datos_modificados data
datos_modificados.to_csv("../data/datos_modificados.csv", index=False)